# Organize all tweets Harvey, Irma and Maria
We need to standard the format of all our tweets because some are JSON and others are .tsv format. LET'S DO DIS.

In [1]:
import pandas as pd
import json
import gzip

Desired format:

- row = tweet
- columns = tweet ID, time created, text, favorites, place, retweets, user name, user id, user place, user bio, retweet data

In [4]:
from datetime import datetime
DATE_FMT = '%a %b %d %H:%M:%S +0000 %Y'
def extract_data_from_json(tweet_data):
    """
    Extract relevant data from json object.
    
    Parameters:
    -----------
    tweet_data : dict
    
    Returns:
    --------
    tweet_data_extract : dict
    """
    tweet_data_extract = {}
    tweet_data_extract['id'] = tweet_data['id']
    tweet_data_extract['created_at'] = datetime.strptime(tweet_data['created_at'], DATE_FMT)
    tweet_data_extract['text'] = tweet_data['text']
    tweet_data_extract['lang'] = tweet_data['lang']
    tweet_data_extract['favorites'] = tweet_data['favorite_count']
    tweet_data_extract['retweets'] = tweet_data['retweet_count']
    if(tweet_data.get('place')):
        tweet_data_extract['place'] = tweet_data['place']['full_name']
        tweet_data_extract['place.type'] = tweet_data['place']['place_type']
    user_data = tweet_data['user']
    tweet_data_extract['user.name'] = user_data['screen_name']
    tweet_data_extract['user.id'] = user_data['id']
    tweet_data_extract['user.bio'] = user_data['description']
    tweet_data_extract['user.followers'] = user_data['followers_count']
    tweet_data_extract['user.friends'] = user_data['friends_count']
    tweet_data_extract['user.lang'] = user_data['lang']
    tweet_data_extract['user.statuses'] = user_data['statuses_count']
    tweet_data_extract['user.place'] = user_data['location']
    if(tweet_data.get('retweeted_status')):
        rt_data = tweet_data['retweeted_status']
        rt_data = extract_data_from_json(rt_data)
        rt_data = {'retweeted.%s'%(k) : v for k,v in rt_data.iteritems()}
        tweet_data_extract.update(rt_data)
    return tweet_data_extract

# Harvey

In [8]:
# start with easy file
doc_now_data_file = '../../data/mined_tweets/HurricaneHarvey_ids_rehydrated.json.gz'
harvey_data_list_1 = []
for i, l in enumerate(gzip.open(doc_now_data_file, 'r')):
    l_json = json.loads(l.strip())
    try:
        l_data = extract_data_from_json(l_json)
        harvey_data_list_1.append(l_data)
    except Exception, e:
        print('error with line %s'%(l_json))
    if(i % 1000000 == 0):
        print('processed %d tweets'%(i))

processed 0 tweets
processed 1000000 tweets
processed 2000000 tweets
processed 3000000 tweets
processed 4000000 tweets
processed 5000000 tweets
processed 6000000 tweets
error with line 911233687156600839
error with line 911233742810775553
error with line 911233811282788352
error with line 911233911723892736
error with line 911234040874848257
error with line 911234342311092229
error with line 911234344743768065
error with line 911234348380295168
error with line 911234356542394369
error with line 911234500239249409
error with line 911234905979449346
error with line 911234991937474561
error with line 911235154303242241
error with line 911235506075299840
error with line 911235788549115904
error with line 911235883831054336
error with line 911235929649684482
error with line 911236042539372545
error with line 911236061413724160
error with line 911236113095970818
error with line 911236342767669248
error with line 911236412388859905
error with line 911236549823664131
error with line 9112369189

In [49]:
# combine
harvey_data_df_1 = pd.concat([pd.Series(t) for t in harvey_data_list_1], axis=1).transpose()

In [ ]:
# now add the individual JSON files (from archive)
import re
import os
from ast import literal_eval
data_dir = '../../data/mined_tweets/'
file_keyword = '#harvey'
file_matcher = re.compile('archive.*%s.*.gz'%(file_keyword))
harvey_json_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if file_matcher.match(f.lower())]
harvey_data_list_2 = []
tweet_keyword = 'harvey'
tweet_matcher = re.compile('.*%s.*'%(tweet_keyword))
tweet_ctr = 0
for f in harvey_json_files:
    for l in gzip.open(f, 'r'):
        # some JSON decoding error...WHY
#         try:
#             l = l.strip().decode('utf-8')
#             l_json = json.loads(l)
#         except Exception, e:
#             print('error at line %s'%(l))
#             break
        l_json = literal_eval(l.strip())
        if(tweet_matcher.match(l_json['text'].lower())):
            l_data = extract_data_from_json(l_json)
            harvey_data_list_2.append(l_data)
            tweet_ctr += 1
            if(len(harvey_data_list_2) % 1000 == 0):
                print('processed %d tweets'%(len(harvey_data_list_2)))

processed 1000 tweets
processed 2000 tweets
processed 3000 tweets
processed 4000 tweets
processed 5000 tweets
processed 6000 tweets
processed 7000 tweets
processed 8000 tweets
processed 9000 tweets
processed 10000 tweets
processed 11000 tweets
processed 12000 tweets
processed 13000 tweets

In [51]:
harvey_data_df_2 = pd.concat([pd.Series(t) for t in harvey_data_list_2], axis=1).transpose()

In [39]:
def convert_historical_df_to_standard(df):
    """
    Fix data columns to match standard.
    """
    date_fmt = '%Y-%m-%d %H:%M'
    df_new = df.copy()
    # fix columns
    df_new.rename(columns={'username' : 'user.name', 'date' : 'created_at', 'geo' : 'place'}, inplace=True)
    df_new.drop(['mentions', 'hashtags','permalink'], axis=1, inplace=True)
    # fix date
    df_new = df_new.assign(created_at=df_new.loc[:, 'created_at'].apply(lambda x: datetime.strptime(x, date_fmt)))
    return df_new

In [40]:
# now add the individual .tsv files...which should really be JSON format for #maximumData
file_matcher = re.compile('(?<!archive)#.*harvey.*.gz')
tweet_keyword = 'harvey'
tweet_matcher = re.compile('.*%s.*'%(tweet_keyword))
harvey_tsv_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if file_matcher.match(f.lower())]
harvey_data_list_3 = [pd.read_csv(f, sep='\t', index_col=None, header=0) for f in harvey_tsv_files]
harvey_data_df_3 = pd.concat(harvey_data_list_3, axis=0)
# remove irrelevant rows 
harvey_data_df_3.dropna(axis=0,subset=['text'],inplace=True)
harvey_data_df_3 = harvey_data_df_3[harvey_data_df_3.loc[:, 'text'].apply(lambda x: tweet_matcher.match(x.lower()) is not None)]
# make sure that the columns match
harvey_data_df_3 = convert_historical_df_to_standard(harvey_data_df_3)

Combine EVERYTHING.

In [ ]:
# harvey_data_combined = pd.concat([harvey_data_df_1, harvey_data_df_2, harvey_data_df_3], axis=0)
# remove duplicates? we have to be careful not to destroy useful data!
print('complete data has %d tweets'%(harvey_data_combined.shape[0]))
harvey_data_deduplicated = harvey_data_combined.drop_duplicates('id', inplace=False)
print('deduplicated data has %d tweets'%(harvey_data_deduplicated.shape[0]))

In [ ]:
print('deduplicated data has %d tweets'%(harvey_data_deduplicated.shape[0]))

In [46]:
print('complete data has %d tweets'%(harvey_data_combined.shape[0]))

complete data has 176985299 tweets


Write to file!!

In [61]:
harvey_data_combined.to_csv('../../data/mined_tweets/harvey_combined.tsv', encoding='utf-8', sep='\t', index=False)

Now do the same thing with the rest of the files.

# Irma

In [ ]:
def load_archive_tweets(data_dir, keyword):
    file_keyword = '#%s'%(keyword)
    file_matcher = re.compile('archive.*%s.*.gz'%(file_keyword))
    json_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if file_matcher.match(f.lower())]
    data_list = []
    tweet_matcher = re.compile('.*%s.*'%(keyword))
    tweet_ctr = 0
    for f in json_files:
        for l in gzip.open(f, 'r'):
            l_json = literal_eval(l.strip())
            if(tweet_matcher.match(l_json['text'].lower())):
                l_data = extract_data_from_json(l_json)
                data_list.append(l_data)
                tweet_ctr += 1
                if(len(data_list) % 1000 == 0):
                    print('processed %d tweets'%(len(data_list)))
    data_df = pd.concat([pd.Series(t) for t in data_list], axis=1).transpose()
    return data_df

In [82]:
def load_historical_tweets(data_dir, keyword):
    file_matcher = re.compile('(?<!archive_)#.*%s.*.gz'%(keyword))
    tweet_matcher = re.compile('.*%s.*'%(keyword))
    tsv_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if file_matcher.match(f.lower())]
    data_list = [pd.read_csv(f, sep='\t', index_col=None, header=0) for f in tsv_files]
    data_df = pd.concat(data_list, axis=0)
    # remove irrelevant rows 
    data_df.dropna(axis=0,subset=['text'],inplace=True)
    data_df = data_df[data_df.loc[:, 'text'].apply(lambda x: tweet_matcher.match(x.lower()) is not None)]
    # make sure that the columns match
    data_df = convert_historical_df_to_standard(data_df)
    return data_df

In [74]:
data_dir = '../../data/mined_tweets/'
keyword = 'irma'
irma_archive_tweets = load_archive_tweets(data_dir, keyword)
irma_historical_tweets = load_historical_tweets(data_dir, keyword)
irma_combined = pd.concat([irma_archive_tweets, irma_historical_tweets], axis=0)

processed 1000 tweets
processed 2000 tweets
processed 3000 tweets
processed 4000 tweets
processed 5000 tweets
processed 6000 tweets
processed 7000 tweets
processed 8000 tweets
processed 9000 tweets
processed 10000 tweets
processed 11000 tweets
processed 12000 tweets
processed 13000 tweets
processed 14000 tweets
processed 15000 tweets
processed 16000 tweets
processed 17000 tweets
processed 18000 tweets
processed 19000 tweets
processed 20000 tweets
processed 21000 tweets
processed 22000 tweets
processed 23000 tweets
processed 24000 tweets
processed 25000 tweets
processed 26000 tweets
processed 27000 tweets
processed 28000 tweets
processed 29000 tweets
processed 30000 tweets
processed 31000 tweets
processed 32000 tweets
processed 33000 tweets
processed 34000 tweets
processed 35000 tweets
processed 36000 tweets
processed 37000 tweets
processed 38000 tweets
processed 39000 tweets
processed 40000 tweets
processed 41000 tweets
processed 42000 tweets
processed 43000 tweets
processed 44000 twee

In [76]:
# save
irma_combined.to_csv('../../data/mined_tweets/irma_combined.tsv', sep='\t', index=False, encoding='utf-8')

# Maria

In [95]:
def load_streaming_data(data_dir, keyword):
    keyword_matcher = re.compile('.*%s_tweets.*'%(keyword))
    data_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if keyword_matcher.match(f)]
    data_list = []
    tweet_matcher = re.compile('.*%s.*'%(keyword))
    tweet_ctr = 0
    for f in data_files:
        for l in gzip.open(f, 'r'):
            try:
                l_json = json.loads(l.strip())
    #             l_json = literal_eval(l.strip())
                if(tweet_matcher.match(l_json['text'].lower())):
                    l_data = extract_data_from_json(l_json)
                    data_list.append(l_data)
                    tweet_ctr += 1
                    if(len(data_list) % 1000 == 0):
                        print('processed %d tweets'%(len(data_list)))
            except Exception, e:
                print('failed to load tweet %s'%(l))
    data_df = pd.concat([pd.Series(t) for t in data_list], axis=1).transpose()
    return data_df

In [83]:
keyword = 'maria'
maria_df_1 = load_historical_tweets(data_dir, keyword)

In [96]:
maria_df_2 = load_streaming_data(data_dir, keyword)

processed 1000 tweets
processed 2000 tweets
processed 3000 tweets
processed 4000 tweets
processed 5000 tweets
processed 6000 tweets
processed 7000 tweets
processed 8000 tweets
processed 9000 tweets
processed 10000 tweets
processed 11000 tweets
processed 12000 tweets
processed 13000 tweets
processed 14000 tweets
processed 15000 tweets
processed 16000 tweets
processed 17000 tweets
processed 18000 tweets
failed to load tweet {"limit":{"track":2,"timestamp_ms":"1506481314460"}}

processed 19000 tweets
processed 20000 tweets
processed 21000 tweets
failed to load tweet {"limit":{"track":2,"timestamp_ms":"1506486842228"}}

failed to load tweet {"limit":{"track":2,"timestamp_ms":"1506486843352"}}

processed 22000 tweets
processed 23000 tweets
processed 24000 tweets
processed 25000 tweets
processed 26000 tweets
processed 27000 tweets
failed to load tweet {"limit":{"track":3,"timestamp_ms":"1506500469026"}}

processed 28000 tweets
processed 29000 tweets
processed 30000 tweets
processed 31000 twe

In [97]:
maria_combined = pd.concat([maria_df_1, maria_df_2], axis=0)
maria_combined.to_csv('../../data/mined_tweets/maria_combined.tsv', sep='\t', index=False, encoding='utf-8')

# Nate

In [99]:
keyword = 'nate'
nate_df_1 = load_historical_tweets(data_dir, keyword)
nate_df_1.to_csv('../../data/mined_tweets/nate_combined.tsv', sep='\t', encoding='utf-8', index=False)

# Ophelia

In [100]:
keyword = 'ophelia'
ophelia_df_1 = load_historical_tweets(data_dir, keyword)
ophelia_df_2 = load_streaming_data(data_dir, keyword)
ophelia_df_combined = pd.concat([ophelia_df_1, ophelia_df_2], axis=0)
ophelia_df_combined.to_csv('../../data/mined_tweets/ophelia_combined.tsv', sep='\t', encoding='utf-8', index=False)

processed 1000 tweets
processed 2000 tweets
processed 3000 tweets
processed 4000 tweets
processed 5000 tweets
processed 6000 tweets
processed 7000 tweets
processed 8000 tweets
processed 9000 tweets
